In [1]:
import math
import numpy as np
import scipy.sparse.linalg # to use tridiagonal solver

### 1-d Heat Equation: $$u_t=\sigma u_{xx}$$

Explicit Finite Difference Scheme $$u_j^{n+1}=u_j^n+\alpha[u_{j+1}^n-2u_j^n+u_{j-1}^n],\ j=1,...,M-1, n=0,...,N-1$$ with $\alpha=\sigma\frac{k}{h^2}$:

In [2]:
def FD(I, L, T, sigma,h,k):
    '''
    I: initial conditions u_0(x)
    L: endpoint
    T: maximum time
    h: Delta x
    k: Delta t
    '''
    M= int((L-0)/h) # M: number of x's
    N= int((T-0)/k) # N: number of t's
    x=np.linspace(0,L,M+1) # mesh points in space
    t=np.linspace(0,T,N+1) # mesh points in time 
    a = sigma*k/(h**2) # a: alpha
    u = np.zeros(M+1) # unknown u at new time level
    u_n = np.zeros(M+1) # u at the previous time level
    u_all=[] # u at all time levels
    # Set initial condition u(x,0)=u_0(x)=I(x)
    for j in range(0,M+1):
        u_n[j]=I(x[j])
    u_all.append(u_n)
    for n in range(0,N):
        # Compute u at inner mesh points
        for j in range(1,M):
            u[j]=u_n[j]+a*(u_n[j+1]-2*u_n[j]+u_n[j-1])
        
        # Insert boundary condition u(0,t)=u(L,t)=0 for this problem
        u[0]=0
        u[M]=0
        
        # Update u_n before the next step
        u_n, u=u, u_n
        u_all.append(u_n.copy())
    
    return u_n,x,t,u_all 
#u_n holds latest u, i.e. u(x,T), u_all holds u at all time levels

Backward Time Difference (implicit Euler) Method:

In [3]:
def BFD(I, L, T, sigma,h,k):
    # I: initial conditions u_0(x)
    # L: endpoint
    # T: maximum time
    # h: Delta x
    # k: Delta t
    M= int((L-0)/h) # M: number of x's
    N= int((T-0)/k) # N: number of t's)
    x=np.linspace(0,L,M+1) # mesh points in space
    t=np.linspace(0,T,N+1) # mesh points in time 
    a = sigma*k/(h**2) # a: alpha
    u = np.zeros(M+1) # unknown u at new time level
    u_n = np.zeros(M+1) # u at the previous time level
    u_all=[] # u at all time levels
    
    # Representation of sparse matrix and right-hand side 
    #  (Using the fact that A is a tridiagonal matrix)
    main = np.zeros(M+1)
    lower = np.zeros(M)
    upper = np.zeros(M)
    b = np.zeros(M+1)
    # Precompute sparse matrix
    main[:]=1+2*a
    lower[:]=-a
    upper[:]=-a
    # Insert boundary conditions
    main[0]=1
    main[M]=1
    upper[0]=0
    lower[M-1]=0
    
    A = scipy.sparse.diags(diagonals=[main,lower,upper],
                          offsets = [0,-1,1], shape=(M+1,M+1), format='csr')
    # print(A.todense()) to check A
    
    # Set initial condition u(x,0)=u_0(x)=I(x)
    for j in range(0,M+1):
        u_n[j]=I(x[j])
    u_all.append(u_n)
    
    for n in range(0,N):
        b=u_n
        b[0]=b[-1]=0.0 # boundary conditions
        u[:]= scipy.sparse.linalg.spsolve(A,b) 
        #u^{n+1}=A^{-1}u^n => A u^{n+1}=u^n
        u_all.append(u.copy())
        u_n[:]=u
    
    return u_n,x,t,u_all

### 2-d Heat Equation: $$u_t=u_{xx}+u_{yy}$$

Alternating Direction Implicit (ADI) Method:

In [4]:
def ADI(I, T, Lx, Ly, Bc, sigma,h,k):
    '''
    I: initial conditions u_0(x,y)
    Lx, Ly: endpoints
    Bc: Boundary condition
    T: maximum time
    h: Delta x & Delta y
    k: Delta t
    '''
    M1= int((Lx-0)/h) # M: number of x's
    M2= int((Ly-0)/h) # M2: number of y's
    N= int((T-0)/k) # N: number of t's
    x=np.linspace(0,Lx,M1+1) # mesh points in x direction
    y=np.linspace(0,Ly,M2+1) # mesh points in y direction
    t=np.linspace(0,T,N+1) # mesh points in time 
    a = sigma*k/(h**2) # a: alpha
    u=np.zeros((M1+1,M2+1)) # temperory solution at new time level t_n
    utemp = u.copy() # intermediate solution u_jl^{n+1/2}
    un =u.copy() #solution u_jl^{n+1}
    u_all=[]
   
    # Representation of matrix A
    #  (Using the fact that A is a tridiagonal matrix)
    main = np.zeros(M1+1)
    lower = np.zeros(M1)
    upper = np.zeros(M1)
    b = np.zeros(M1+1)
    # Precompute sparse matrix
    main[:]=2+2*a
    lower[:]=-a
    upper[:]=-a
    # Insert boundary conditions
    main[0]=2
    main[M1]=2
    upper[0]=0
    lower[M1-1]=0
    
    A = scipy.sparse.diags(diagonals=[main,lower,upper],
                          offsets = [0,-1,1], shape=(M1+1,M1+1), format='csr')
    
    # Representation of matrix B
    #  (Using the fact that B is a tridiagonal matrix)
    main = np.zeros(M2+1)
    lower = np.zeros(M2)
    upper = np.zeros(M2)
    c = np.zeros(M2+1)
    # Precompute sparse matrix
    main[:]=2+2*a
    lower[:]=-a
    upper[:]=-a
    # Insert boundary conditions
    main[0]=2
    main[M2]=2
    upper[0]=0
    lower[M2-1]=0
    
    B = scipy.sparse.diags(diagonals=[main,lower,upper],
                          offsets = [0,-1,1], shape=(M2+1,M2+1), format='csr')
    
     # Set initial condition u(0,x,y)=I(x,y)
    for j in range(0,M1+1):
        for l in range(0,M2+1):
            u[j,l]=I(x[j],y[l])
    #print(u)
    u_all.append(deepcopy(u))

    for n in range(0,N):
        # Compute u at inner mesh points
        for j in range(0,M1+1):
            utemp[j,0]=Bc(x[j],y[0],t[n]+k/2)
            utemp[j,M2] = Bc(x[j], y[M2], t[n]+k/2)
        for l in range(1,M2):
            # first treat lower boundary for column l:
            b[0] = Bc(x[0], y[l], t[n]+k/2)
            # treat upper boundary:
            b[M1] = Bc(x[M1], y[l], t[n]+k/2)
            
            # run through all inner points for column l:
            for j in range(1,M1):
                b[j] = 2*u[j,l] + a*(u[j,l-1] - 2*u[j,l] + u[j,l+1])
            
            # solve linear system:
            tmp =  scipy.sparse.linalg.spsolve(A,b)
            
            # insert solution into column l:
            for j in range(0,M1+1):
                utemp[j,l] = tmp[j]
        
        # y-direction sweep:
        # boundary
        for l in range(0,M2+1):
            un[0,l] = Bc(x[0], y[l], t[n]+k)
            un[M1,l] = Bc(x[M1], y[l], t[n]+k)
        # solve linear tridiagonal system for all internal rows j:
        for j in range(1,M1):
            # first treat left boundary for row j:
            c[0] = Bc(x[j], y[0], t[n]+k)
            # treat right boundary
            c[M2] = Bc(x[j], y[M2], t[n]+k)
            # then run through all inner points for row j:
            for l in range(1,M2):
                c[l] = 2*utemp[j,l] + a*(utemp[j-1,l] - 2*utemp[j,l] + utemp[j+1,l])
            
            # solve linear system:
            tmp2 = scipy.sparse.linalg.spsolve(B,c)
            # insert solution into row j:
            for l in range(0,M2+1):
                un[j,l] = tmp2[l]            
        
        # update data structures for next step:
        u, un = un, u
        #print(un[3][1],t[n]+k)
        u_all.append(deepcopy(u))
    #print(utemp)
    return u,u_all,x,y,t
#u holds latest solution, i.e. u(T,x,y), u holds solution at all time levels